<a href="https://colab.research.google.com/github/shadyarmmu/mmu-hpc/blob/main/Final_FlexMatch_Conf95_CrossSplit_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!unzip -q -P unlocked308BR /content/gdrive/MyDrive/DeepLearning/DFUC2021_trainset_210427.zip -d dfu_train

In [ ]:
!unzip -q -P sigmoid608KL /content/gdrive/MyDrive/DeepLearning/DFUC2021_testing_release.zip -d dfu_test

In [ ]:
#installing torch
!pip install lightning-utilities
!pip install torchmetrics --no-deps

#Importing the libries

In [ ]:

import pandas as pd
import numpy as np
import os
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import torch.nn.functional as F

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

print("PyTorch Version:", torch.__version__)
print("Libraries imported successfully!")

PyTorch Version: 2.6.0+cu124
Libraries imported successfully!


# Estabulishing Global variables

In [ ]:
BATCH_SIZE = 16
UNLABELED_BATCH_SIZE = BATCH_SIZE * 7
IMG_HEIGHT = 224
IMG_WIDTH = 224
CONFIDENCE_THRESHOLD = 0.85
SEED = 42
N_SPLITS = 4 # Number of folds for cross-validation
NUM_EPOCHS = 50 # Number of epochs to train for EACH fold
LAMBDA_U = 1.0  # Weight for the consistency loss

#Setting seeds
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# Set device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

Using device: cuda


# Loading the data and K-Folding

In [ ]:
TRAIN_IMG_DIR = "/content/dfu_train/DFUC2021_train/images"
CSV_FILE = "/content/dfu_train/DFUC2021_train/train.csv"
CLASS_NAMES = ['none', 'infection', 'ischaemia', 'both']
NUM_CLASSES = len(CLASS_NAMES)

# Loadingg the training data metadata
df = pd.read_csv(CSV_FILE)
print("--- Successfully loaded train.csv ---")

# Separating labeled and unlabeled data
labeled_df = df[df['none'].notna()].copy()
unlabeled_df = df[df['none'].isna()][['image']].copy()

print(f"\nTotal examples: {len(df)}")
print(f"Number of labeled examples to be split: {len(labeled_df)}")
print(f"Number of unlabeled examples for training: {len(unlabeled_df)}")

# Converting one-hot encoded labels to a single integer label column
labeled_df['label'] = np.argmax(labeled_df[CLASS_NAMES].values, axis=1)

# Seting up K-Fold Cross-Validation
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
print(f"\nInitialized StratifiedKFold with {N_SPLITS} splits.")

--- Successfully loaded train.csv ---

Total examples: 9949
Number of labeled examples to be split: 5955
Number of unlabeled examples for training: 3994

Initialized StratifiedKFold with 4 splits.


# dataset augmentation and creating custom datasets

In [ ]:

# Image Transformations and
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transform_weak = transforms.Compose([
    transforms.Resize((IMG_WIDTH + 30, IMG_HEIGHT + 30)),
    transforms.RandomCrop((IMG_HEIGHT, IMG_WIDTH)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

transform_strong = transforms.Compose([
    transforms.Resize((IMG_WIDTH + 30, IMG_HEIGHT + 30)),
    transforms.RandomCrop((IMG_HEIGHT, IMG_WIDTH)),
    transforms.RandomHorizontalFlip(),
    transforms.RandAugment(num_ops=2, magnitude=10),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

transform_val = transforms.Compose([
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# Custom Dataset Classes
class LabeledDFUCDataset(Dataset):
    """Dataset for labeled data. Returns (image, label) pairs."""
    def __init__(self, df, img_dir, transform):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        self.image_files = self.df['image'].values
        self.labels = self.df['label'].values

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.image_files[idx])
        image = Image.open(img_path).convert("RGB")
        image = self.transform(image)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return image, label

class UnlabeledDFUCDataset(Dataset):
    """Dataset for unlabeled data. Returns (weakly_augmented, strongly_augmented) pairs."""
    def __init__(self, df, img_dir, weak_transform, strong_transform):
        self.df = df
        self.img_dir = img_dir
        self.weak_transform = weak_transform
        self.strong_transform = strong_transform
        self.image_files = self.df['image'].values

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.image_files[idx])
        image = Image.open(img_path).convert("RGB")
        img_weak = self.weak_transform(image)
        img_strong = self.strong_transform(image)
        return img_weak, img_strong

# Model Creation Function
def create_model(num_classes):
    """Creates a pre-trained EfficientNet-B1 model with a custom classifier."""
    model = models.efficientnet_b1(weights=models.EfficientNet_B1_Weights.IMAGENET1K_V1)
    for param in model.parameters():
        param.requires_grad = False

    num_ftrs = model.classifier[1].in_features
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.2, inplace=True),
        nn.Linear(num_ftrs, num_classes)
    )
    return model

# Loss Functions
supervised_loss_fn = nn.CrossEntropyLoss()
consistency_loss_fn = nn.MSELoss()


#The Loop

In [ ]:
fold_histories = {}
best_fold_accuracies = {}

# Main K-Fold Loop
for fold, (train_idx, val_idx) in enumerate(skf.split(labeled_df, labeled_df['label'])):
    print(f"\n{'='*25} FOLD {fold + 1}/{N_SPLITS} {'='*25}")

    # 1. Create Data for the Current Fold
    train_labeled_df = labeled_df.iloc[train_idx]
    val_df = labeled_df.iloc[val_idx]

    print(f"Labeled training set size: {len(train_labeled_df)}")
    print(f"Validation set size: {len(val_df)}")

    # Create Datasets and DataLoaders
    train_labeled_dataset = LabeledDFUCDataset(train_labeled_df, TRAIN_IMG_DIR, transform_weak)
    train_unlabeled_dataset = UnlabeledDFUCDataset(unlabeled_df, TRAIN_IMG_DIR, transform_weak, transform_strong)
    val_dataset = LabeledDFUCDataset(val_df, TRAIN_IMG_DIR, transform_val)

    train_labeled_loader = DataLoader(train_labeled_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
    train_unlabeled_loader = DataLoader(train_unlabeled_dataset, batch_size=UNLABELED_BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

    # 2. Re-initialize Model, Optimizer, and Scheduler for each Fold
    print("\nRe-initializing model for new fold...")
    model = create_model(num_classes=NUM_CLASSES)
    model.to(DEVICE)

    trainable_params = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = optim.AdamW(trainable_params, lr=3e-4, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS, eta_min=1e-6)

    # 3. Run the Training and Validation Loop
    BEST_MODEL_PATH = f"best_model_fold_{fold+1}.pth"
    best_val_accuracy = 0.0
    history = {'train_loss': [], 'val_loss': [], 'val_acc': [], 'val_f1': []}

    for epoch in range(NUM_EPOCHS):
        print(f"\n--- Epoch {epoch+1}/{NUM_EPOCHS} ---")
        model.train()
        total_loss = 0
        unlabeled_iter = iter(train_unlabeled_loader)
        progress_bar = tqdm(train_labeled_loader, desc=f"Training Fold {fold+1}")

        for batch_idx, (labeled_images, true_labels) in enumerate(progress_bar):
            try:
                unlabeled_weak, unlabeled_strong = next(unlabeled_iter)
            except StopIteration:
                unlabeled_iter = iter(train_unlabeled_loader)
                unlabeled_weak, unlabeled_strong = next(unlabeled_iter)

            labeled_images, true_labels = labeled_images.to(DEVICE), true_labels.to(DEVICE)
            unlabeled_weak, unlabeled_strong = unlabeled_weak.to(DEVICE), unlabeled_strong.to(DEVICE)

            # Supervised Loss
            labeled_preds = model(labeled_images)
            loss_sup = supervised_loss_fn(labeled_preds, true_labels)

            # Consistency Loss (FlexMatch)
            with torch.no_grad():
                unlabeled_weak_preds = model(unlabeled_weak)
                pseudo_label_probs = torch.softmax(unlabeled_weak_preds, dim=1)
                max_probs, pseudo_labels = torch.max(pseudo_label_probs, dim=1)
                mask = max_probs.ge(CONFIDENCE_THRESHOLD).float()

            unlabeled_strong_preds = model(unlabeled_strong)
            loss_unsup = (F.cross_entropy(unlabeled_strong_preds, pseudo_labels, reduction='none') * mask).mean()

            # Combine Losses and Backpropagate
            total_batch_loss = loss_sup + (LAMBDA_U * loss_unsup)
            optimizer.zero_grad()
            total_batch_loss.backward()
            optimizer.step()

            total_loss += total_batch_loss.item()
            progress_bar.set_postfix({'Loss': f"{total_batch_loss.item():.4f}", 'Mask': f"{mask.mean().item():.2f}"})

        scheduler.step()
        avg_train_loss = total_loss / len(train_labeled_loader)
        history['train_loss'].append(avg_train_loss)

        # Validation Phase
        model.eval()
        val_loss, all_preds, all_labels = 0, [], []
        with torch.no_grad():
            for val_images, val_labels in val_loader:
                val_images, val_labels = val_images.to(DEVICE), val_labels.to(DEVICE)
                outputs = model(val_images)
                loss = supervised_loss_fn(outputs, val_labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(val_labels.cpu().numpy())

        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = accuracy_score(all_labels, all_preds)
        val_f1 = f1_score(all_labels, all_preds, average='macro')
        history['val_loss'].append(avg_val_loss)
        history['val_acc'].append(val_accuracy)
        history['val_f1'].append(val_f1)

        print(f"Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | Val Acc: {val_accuracy:.4f} | Val F1: {val_f1:.4f}")

        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            torch.save(model.state_dict(), BEST_MODEL_PATH)
            print(f"🎉 New best model for fold {fold+1} saved to {BEST_MODEL_PATH} with accuracy: {best_val_accuracy:.4f}")

    fold_histories[f"fold_{fold+1}"] = history
    best_fold_accuracies[f"fold_{fold+1}"] = best_val_accuracy
    print(f"Finished training for fold {fold+1}. Best validation accuracy: {best_val_accuracy:.4f}")

# --- Final Summary ---
print("\n\n" + "="*55)
print("✅ CROSS-VALIDATION TRAINING COMPLETE")
print("="*55)
for fold, acc in best_fold_accuracies.items():
    print(f"Best Validation Accuracy for {fold}: {acc:.4f}")

mean_accuracy = np.mean(list(best_fold_accuracies.values()))
print(f"\nAverage Best Validation Accuracy across all folds: {mean_accuracy:.4f}")




========================= FOLD 1/4 =========================
Labeled training set size: 4466
Validation set size: 1489

Re-initializing model for new fold...

--- Epoch 1/50 ---


Training Fold 1: 100%|██████████| 280/280 [01:43<00:00,  2.69it/s, Loss=0.8922, Mask=0.01]


Train Loss: 0.9937 | Val Loss: 0.9314 | Val Acc: 0.6158 | Val F1: 0.4437
🎉 New best model for fold 1 saved to best_model_fold_1.pth with accuracy: 0.6158

--- Epoch 2/50 ---


Training Fold 1: 100%|██████████| 280/280 [01:43<00:00,  2.71it/s, Loss=0.8917, Mask=0.05]


Train Loss: 0.8611 | Val Loss: 0.8667 | Val Acc: 0.6347 | Val F1: 0.5022
🎉 New best model for fold 1 saved to best_model_fold_1.pth with accuracy: 0.6347

--- Epoch 3/50 ---


...

Train Loss: 0.7862 | Val Loss: 0.7106 | Val Acc: 0.6989 | Val F1: 0.7041

--- Epoch 49/50 ---


Training Fold 4: 100%|██████████| 280/280 [01:43<00:00,  2.70it/s, Loss=1.8596, Mask=0.27]


Train Loss: 0.7835 | Val Loss: 0.7120 | Val Acc: 0.7050 | Val F1: 0.7024

--- Epoch 50/50 ---


Training Fold 4: 100%|██████████| 280/280 [01:43<00:00,  2.71it/s, Loss=1.0333, Mask=0.21]


Train Loss: 0.7832 | Val Loss: 0.7131 | Val Acc: 0.7023 | Val F1: 0.6931
Finished training for fold 4. Best validation accuracy: 0.7103


✅ CROSS-VALIDATION TRAINING COMPLETE
Best Validation Accuracy for fold_1: 0.6884
Best Validation Accuracy for fold_2: 0.6884
Best Validation Accuracy for fold_3: 0.6723
Best Validation Accuracy for fold_4: 0.7103

Average Best Validation Accuracy across all folds: 0.6898


# Creating prediction CSV file for DFUC2021 evaluation

In [ ]:
print("\n\n" + "="*55)
print("🚀 STARTING PREDICTION ON TEST SET")
print("="*55)

TEST_IMG_DIR = '/content/dfu_test/DFUC2021_test'
OUTPUT_CSV_PATH = 'submission_ensemble_4-fold.csv'
MODEL_PATHS = [f"best_model_fold_{i}.pth" for i in range(1, N_SPLITS + 1)]

print("--- Prediction Script using 4-Fold Ensemble (Averaging Probabilities) ---")
print(f"Models to be used for ensemble: {MODEL_PATHS}")

# Create Test Dataset and DataLoader
class TestDataset(Dataset):
    def __init__(self, image_files, img_dir, transform):
        self.image_files = image_files
        self.img_dir = img_dir
        self.transform = transform
    def __len__(self):
        return len(self.image_files)
    def __getitem__(self, idx):
        fname = self.image_files[idx]
        img_path = os.path.join(self.img_dir, fname)
        image = Image.open(img_path).convert("RGB")
        image = self.transform(image)
        return image, fname

transform_test = transforms.Compose([
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

if not os.path.isdir(TEST_IMG_DIR):
    print(f"ERROR: Test image directory not found at '{TEST_IMG_DIR}'")
else:
    test_image_files = sorted([f for f in os.listdir(TEST_IMG_DIR) if f.endswith(('.jpg', '.jpeg', '.png'))])
    print(f"Found {len(test_image_files)} images in the test directory.")
    test_dataset = TestDataset(test_image_files, TEST_IMG_DIR, transform_test)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

    # Generate Predictions from Each Model
    all_fold_probs = []
    for fold_num, model_path in enumerate(MODEL_PATHS):
        print(f"\n--- Generating predictions with model from Fold {fold_num + 1} ---")
        model = create_model(num_classes=NUM_CLASSES)
        try:
            model.load_state_dict(torch.load(model_path, map_location=DEVICE))
        except FileNotFoundError:
            print(f"ERROR: Model file not found at {model_path}. Skipping this fold.")
            continue

        model.to(DEVICE)
        model.eval()
        fold_probs = []
        with torch.no_grad():
            for images, fnames in tqdm(test_loader):
                images = images.to(DEVICE)
                outputs = model(images)
                probs = torch.softmax(outputs, dim=1)
                fold_probs.append(probs.cpu().numpy())
        all_fold_probs.append(np.concatenate(fold_probs, axis=0))

    # Average the Probabilities and Create Submission File
    if not all_fold_probs:
        print("No predictions were generated. Exiting.")
    else:
        print("\n✨ Averaging probabilities across all folds...")
        avg_probs = np.mean(all_fold_probs, axis=0)
        submission_df = pd.DataFrame()
        submission_df['image'] = test_image_files
        for i, class_name in enumerate(CLASS_NAMES):
            submission_df[class_name] = avg_probs[:, i]

        submission_df.to_csv(OUTPUT_CSV_PATH, index=False)
        print(f"\n--- Prediction Complete! ---")
        print(f"Ensemble submission file created at: {OUTPUT_CSV_PATH}")
        print("\nFirst 5 rows of the submission file:")
        print(submission_df.head())



🚀 STARTING PREDICTION ON TEST SET
--- Prediction Script using 4-Fold Ensemble (Averaging Probabilities) ---
Models to be used for ensemble: ['best_model_fold_1.pth', 'best_model_fold_2.pth', 'best_model_fold_3.pth', 'best_model_fold_4.pth']
Found 5734 images in the test directory.

--- Generating predictions with model from Fold 1 ---


100%|██████████| 359/359 [00:07<00:00, 44.94it/s]



--- Generating predictions with model from Fold 2 ---


100%|██████████| 359/359 [00:07<00:00, 47.97it/s]



--- Generating predictions with model from Fold 3 ---


100%|██████████| 359/359 [00:07<00:00, 47.46it/s]



--- Generating predictions with model from Fold 4 ---


100%|██████████| 359/359 [00:08<00:00, 44.49it/s]


✨ Averaging probabilities across all folds...

--- Prediction Complete! ---
Ensemble submission file created at: submission_ensemble_4-fold.csv

First 5 rows of the submission file:
        image      none  infection  ischaemia      both
0  501000.jpg  0.153091   0.492121   0.033228  0.321560
1  501001.jpg  0.058244   0.060292   0.645323  0.236142
2  501002.jpg  0.749864   0.232756   0.008151  0.009230
3  501003.jpg  0.729455   0.260079   0.007775  0.002691
4  501004.jpg  0.178082   0.693760   0.032411  0.095748
